Neural Network from scratch

In [ ]:
#Some libraries which need to be installed ( might need restarting of kernel after installing)
! pip3 install ipywidgets --user
! pip install tensorflow
! pip install tensorflow-datasets

In [1]:
# import the necessary packages
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [2]:
# Neural Network from scratch
class NeuralNetwork:
    def __init__(self, layers, alpha=0.001, batchsize=32):
        # initialize the list of weights, l=number of layers, alpha = learning rate, m = batchsize
        self.W = []
        self.layers = layers
        self.alpha = alpha
        self.m = batchsize
        # We randomly initialize the weights of all layers except last 2
        # This is because in last two layers the 2nd last layer has bias term but last layer doesnt have it
        for i in np.arange(0, len(layers) - 2):
            # add an extra node for bias
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.W.append(w / np.sqrt(layers[i]))
        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.W.append(w / np.sqrt(layers[-2]))
    
    def relu(self, x):
        #ReLu function
        return np.maximum(x, 0)
    
    def relu_deriv(self, x):
        # computes derivative of ReLu function
        return (x > 0) * 1
    
    def fit(self, X, y, epochs=1000):
        # We insert a column of ones in features, this for the bias term being uses a a weight
        train_loss = []
        X = np.c_[X, np.ones((X.shape[0]))]
        for epoch in np.arange(0, epochs):
            # We divide the training data into batches
            for i in range(0, X.shape[0], self.m):
                if(i+self.m < X.shape[0]):
                    x = X[i:i+self.m]
                    #print(x)
                    target = y[i:i+self.m]
                else:
                    x = X[i:X.shape[0]]
                    target = y[i:y.shape[0]]
                self.fit_partial(x,target)
            loss = self.cross_entropy(X, y)
            train_loss.append(loss)
            # Print the epoch loss
            print("Epoch={}, loss={:.7f}".format( epoch + 1, loss))
        return train_loss
    
    def fit_partial(self, x, y):
        # This function does the forward and backward pass on a batch
        # We make A as a nested list
        # this will containt the outputs of each layer for each data point in the batch
        A = [[] for _ in range(self.m)]
        for i in range(self.m):
            A[i].append(np.atleast_2d(x[i]))
        #Forward pass
        for i in range(self.m):
            for layer in np.arange(0, len(self.W)):
                net = A[i][layer].dot(self.W[layer])
                if(layer != len(self.W) - 1):
                    out = self.relu(net)            
                else:
                    out = self.softmax(net)    
                A[i].append(out)
        # BACKPROPAGATION
        # We calculate deltas and store it in D
        D = [[] for _ in range(self.m)]
        #Delta of last layer
        for i in range(self.m):
            D[i] = [A[i][-1] - y[i]]
            #print(D[i])
        # Now we apply chain rule and compute delta of all layers
        for i in range(self.m):
            for layer in np.arange(len(A[i]) - 2, 0, -1):
                # Delta of present layer is delta of previous layer dotted by weight
                delta = D[i][-1].dot(self.W[layer].T)
                delta = delta * self.relu_deriv(A[i][layer])
                D[i].append(delta)
        #As the above for loop went reverse, Delta are also stored in reverse manner
        #Hence we reverse it
        for i in range(self.m):
            D[i] = D[i][::-1]
        # Updating the weights
        for layer in np.arange(0, len(self.W)):
            #avg_d sums the gradients over the mini-batch
            avg_d = 0
            for i in range(self.m):
                avg_d = avg_d + A[i][layer].T.dot(D[i][layer])
            #Update the weights
            self.W[layer] += -self.alpha * avg_d/self.m
            
    def softmax(self, predictions):
        #softmax function
        exps = np.exp(predictions - np.max(predictions,axis=1).reshape(-1,1))
        return exps / np.sum(exps,axis=1)[:,None]
    
    def softmax_deriv(self,pred):
        # derivative of softmax
        return pred * (1 -(1 * pred).sum(axis=1)[:,None])
    
    def predict(self, X, addBias=True):
        # obtain the final prediction
        p = np.atleast_2d(X)
        # check to see if the bias column should be added
        if addBias:
            # insert a column of 1's for bias node
            p = np.c_[p, np.ones((p.shape[0]))]
        # loop over our layers in the network
        for layer in np.arange(0, len(self.W)):
            # computing the output prediction
            p = np.dot(p, self.W[layer])
            if(layer != len(self.W) - 1):
                p = self.relu(p)
            else:
                p = self.softmax(p)
        # return the predicted value
        return p
    
    def cross_entropy(self, X, targets):
        #Function to compute the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        predictions = predictions.clip(min=1e-16,max=None)
        loss = 0
        for i in range(predictions.shape[0]):
            loss = loss + np.sum(np.where(targets[i]==1,-np.log(predictions[i]), 0))
        # return the loss
        loss = loss/(predictions.shape[0])
        return loss

    def cross_ent_deriv(self, predictions, y):
        #Function to compute derivative of cross entropy
        predictions = predictions.clip(min=1e-16,max=None)
        return np.where(y==1,-1/predictions, 0)

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
# load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

#Apply label binarizer to convert integers to vectors
y_train_scratch = LabelBinarizer().fit_transform(y_train)
y_test_scratch = LabelBinarizer().fit_transform(y_test)

# train the network
print("Starting training the network...")
nn = NeuralNetwork([x_train.shape[1], 512, 256, 10])
train_loss = nn.fit(x_train, y_train_scratch, epochs=5)

# evaluate the network
predictions = nn.predict(x_test)
print(predictions[0])
predictions = predictions.argmax(axis=1)
print(classification_report(y_test_scratch.argmax(axis=1), predictions))

11501568/11490434 [==============================] - 0s 0us/step
Starting training the network...
Epoch=1, loss=1.5644252
Epoch=2, loss=0.8494830
Epoch=3, loss=0.6021411
Epoch=4, loss=0.4972492
Epoch=5, loss=0.4396602
[9.75070372e-04 3.93704436e-05 3.71597209e-04 2.48399448e-03
 4.51243482e-04 6.57013817e-04 2.03634489e-05 9.80580943e-01
 3.66986910e-04 1.40534172e-02]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       980
           1       0.95      0.97      0.96      1135
           2       0.91      0.84      0.87      1032
           3       0.88      0.89      0.89      1010
           4       0.88      0.90      0.89       982
           5       0.87      0.81      0.84       892
           6       0.91      0.91      0.91       958
           7       0.91      0.89      0.90      1028
           8       0.84      0.85      0.84       974
           9       0.85      0.87      0.86      1009

    accuracy                    

Implementing using Standard Library

In [4]:
Net = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer((784)),
  tf.keras.layers.Dense(512, activation='relu'), 
  tf.keras.layers.Dense(256, activation='relu'),    
  tf.keras.layers.Dense(10, activation = 'softmax')
])

In [5]:
# Training the model
Net.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.0, nesterov=False, name='SGD'),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
Net.fit(x_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 1.6049 - accuracy: 0.6399 - val_loss: 1.0135 - val_accuracy: 0.8092
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.7764 - accuracy: 0.8384 - val_loss: 0.5908 - val_accuracy: 0.8690
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5364 - accuracy: 0.8722 - val_loss: 0.4610 - val_accuracy: 0.8871
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4450 - accuracy: 0.8855 - val_loss: 0.3983 - val_accuracy: 0.8981
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3965 - accuracy: 0.8940 - val_loss: 0.3623 - val_accuracy: 0.9044


In [6]:
# Evaluate the model on the test data
print("Evaluate on test data")
results = Net.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 0s 5ms/step - loss: 0.3623 - accuracy: 0.9044
test loss, test acc: [0.3623420298099518, 0.9043999910354614]
